# Pipeline Example

We will cover the following Topics
1. Pipeline Example
2. Leverage
3. Hedging
4. PyFolio for Portfolioio Analysis
5. Stock Sentiment Trading Strategy
6. Futures

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [2]:
# Stock universes try to avoid not traded error
from quantopian.pipeline.filters import Q1500US

In [3]:
universe = Q1500US()

In [4]:
from quantopian.pipeline.data import morningstar

In [5]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [6]:
#from quantopian.pipeline.classifiers.morningstar import Sector

In [7]:
# Link for different sector codes in notebook
energy_sector = sector.eq(309)

In [8]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [11]:
# Past 30 days
dollar_volume = AverageDollarVolume(window_length=30)

In [19]:
dollar_volume.top(5)
dollar_volume.bottom(5)
# Top 10% of stocks
high_dollar_volume = dollar_volume.percentile_between(90,100)

In [21]:
top_open_prices = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

In [22]:
high_close_price = USEquityPricing.close.latest.percentile_between(90,100,mask= top_open_prices)

In [24]:
def make_pipeline():
    # Universe of Q1500
    base_universe = Q1500US()
    
    # Energy Sector
    energy_sector = sector.eq(309)
    
    # Make a mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar volume (30 days) Grab the iniformation
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[
        USEquityPricing.close
    ],window_length=10,mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[
        USEquityPricing.close
    ],window_length=30,mask=top_five_base_energy)
    
    # Percent difference between
    percent_diff = (mean_10-mean_30) / mean_30
    
    # List of shorts
    shorts = percent_diff < 0
    
    # List of Longs
    longs = percent_diff > 0
    
    # Final mask/Filter for anything in shorts or longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(
        columns ={
            'longs': longs,
            'shorts': shorts,
            'percent_diff': percent_diff,
        },
        screen=securities_to_trade
    )
    

In [25]:
result = run_pipeline(make_pipeline(),'2015-05-05','2015-05-05')

In [26]:
result.head()

longs  percent_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])   True      0.036528  False
                          Equity(448 [APA])   True      0.035926  False
                          Equity(455 [APC])   True      0.049153  False
                          Equity(858 [BHI])   True      0.033807  False
                          Equity(1746 [COG])  True      0.058078  False

In [27]:
len(result)

38

In [28]:
result

longs  percent_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True      0.036528  False
                          Equity(448 [APA])     True      0.035926  False
                          Equity(455 [APC])     True      0.049153  False
                          Equity(858 [BHI])     True      0.033807  False
                          Equity(1746 [COG])    True      0.058078  False
                          Equity(2368 [DVN])    True      0.046264  False
                          Equity(2564 [EOG])    True      0.032102  False
                          Equity(2621 [VAL])    True      0.060197  False
                          Equity(3443 [HAL])    True      0.049257  False
                          Equity(3647 [HP])     True      0.040991  False
                          Equity(5035 [MRO])    True      0.061598  False
                          Equity(5213 [NBL])    True      0.010443  False
                          Equity(5214 [NBR])    True      0.064133  False
                          Equity(5249 [NE])     True      0.037559  False
                          Equity(5729 [OXY])    True      0.029776  False
                          Equity(6928 [SLB])    True      0.046555  False
                          Equity(7244 [SWN])    True      0.070788  False
                          Equity(7612 [ANDV])   True      0.005997  False
                          Equity(7990 [VLO])   False     -0.017145   True
                          Equity(8214 [WMB])    True      0.018876  False
                          Equity(8347 [XOM])    True      0.017343  False
                          Equity(8461 [CHK])    True      0.014265  False
                          Equity(9038 [RIG])    True      0.048180  False
                          Equity(13176 [CAM])   True      0.082110  False
                          Equity(17436 [PXD])   True      0.010248  False
                          Equity(19249 [RRC])   True      0.087062  False
                          Equity(19336 [WFT])   True      0.049141  False
                          Equity(22784 [FTI])   True      0.054529  False
                          Equity(23112 [CVX])   True      0.018972  False
                          Equity(23998 [COP])   True      0.023902  False
                          Equity(24809 [NOV])   True      0.024940  False
                          Equity(25707 [WLL])   True      0.048205  False
                          Equity(33856 [CLR])   True      0.064304  False
                          Equity(34440 [CXO])   True      0.042184  False
                          Equity(39797 [OAS])   True      0.042388  False
                          Equity(40852 [KMI])   True      0.023016  False
                          Equity(41636 [MPC])   True      0.011952  False
                          Equity(42788 [PSX])   True      0.020911  False